In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('clean.csv')

In [3]:
test = pd.read_csv('clean_test.csv')

In [4]:
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)
# target=train.pop('status_group')

In [5]:
train['train']=1
test['train']=0

In [6]:
combined = pd.concat([train,test])

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined['status_group'] = pd.factorize(combined['status_group'])[0]

In [8]:
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

train = train_df.sample(frac=0.8)
validate = train_df.sample(frac=0.1)
test = train_df.sample(frac=0.1)

[xsize, ysize] =train.shape

train_target = train.pop('status_group')
validate_target = validate.pop('status_group')
test_target = test.pop('status_group')

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.models import Model
from keras.utils import to_categorical

num_labels = len(np.unique(train_target))
train_target_cat = to_categorical(train_target)
validate_target_cat = to_categorical(validate_target)
test_target_cat = to_categorical(test_target)

batch_size = 128
hidden_units = 256
dropout = 0.1
input_size = (1 * ysize-1 * 1)

model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))


Using TensorFlow backend.
W0318 18:01:08.988085 140681994278720 deprecation.py:506] From /home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train, train_target_cat,  validation_data=(validate, validate_target_cat), epochs=100, batch_size=batch_size)
score = model.evaluate(test, test_target_cat, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               5120      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
__________

Epoch 47/100
47520/47520 [==============================] - 1s 16us/step - loss: 0.1505 - acc: 0.6623 - val_loss: 0.1446 - val_acc: 0.6810
Epoch 48/100
47520/47520 [==============================] - 1s 16us/step - loss: 0.1508 - acc: 0.6595 - val_loss: 0.1478 - val_acc: 0.6670
Epoch 49/100
47520/47520 [==============================] - 1s 16us/step - loss: 0.1502 - acc: 0.6615 - val_loss: 0.1462 - val_acc: 0.6714
Epoch 50/100
47520/47520 [==============================] - 1s 16us/step - loss: 0.1499 - acc: 0.6642 - val_loss: 0.1445 - val_acc: 0.6811
Epoch 51/100
47520/47520 [==============================] - 1s 16us/step - loss: 0.1501 - acc: 0.6625 - val_loss: 0.1445 - val_acc: 0.6860
Epoch 52/100
47520/47520 [==============================] - 1s 17us/step - loss: 0.1498 - acc: 0.6667 - val_loss: 0.1482 - val_acc: 0.6732
Epoch 53/100
47520/47520 [==============================] - 1s 17us/step - loss: 0.1502 - acc: 0.6620 - val_loss: 0.1439 - val_acc: 0.6832
Epoch 54/100
47520/47520 [=

In [39]:
meow = model.predict_classes(test)
for i in range(meow.shape[-1]):
    print(meow[i])

0
1
0
0
1
1
0
0
1
1
0
1
1
1
0
0
0
0
1
0
1
0
1
0
0
0
1
0
0
0
1
0
1
0
1
1
1
0
0
1
1
0
0
1
0
1
1
1
0
0
1
0
2
0
0
1
1
1
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
0
0
1
0
0
1
1
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
1
0
0
1
1
0
1
1
0
0
1
1
0
1
0
0
0
1
1
0
0
0
0
0
1
1
0
1
0
0
1
0
1
1
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
1
1
1
0
0
0
1
1
0
1
0
1
0
0
1
1
1
0
1
0
1
0
1
0
1
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
1
0
0
1
1
1
0
1
1
1
0
1
1
0
0
1
1
0
0
0
0
0
0
1
0
1
0
0
0
1
1
0
0
0
1
1
0
1
0
0
0
1
0
1
1
0
1
0
0
0
0
0
0
2
0
0
0
0
0
1
0
0
1
0
1
0
1
0
1
0
1
1
0
0
1
0
0
1
1
0
0
0
1
1
0
1
0
0
0
0
1
0
2
0
0
1
1
1
1
1
1
0
0
0
0
1
0
1
0
1
0
1
1
0
0
1
0
0
0
1
0
0
0
1
1
0
1
0
0
0
1
1
0
0
1
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
1
0
0
0
1
0
1
1
1
1
1
1
0
1
0
0
1
1
1
0
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
1
1
1
0
0
0
2
0
0
0
0
0
1
0
1
0
0
1
1
0
1
0
1
1
0
0
1
0
0
1
2
1
1
0
0
0


0
1
0
1
0
0
0
0
1
1
0
0
1
2
0
0
1
0
1
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
2
0
0
1
1
1
1
0
0
1
0
0
0
1
0
1
1
2
1
0
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
0
1
0
0
0
1
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
1
1
0
2
0
1
1
0
1
0
1
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
2
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
1
1
1
0
0
0
0
0
0
0
1
1
1
1
0
1
0
1
1
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
1
0
0
0
0
1
0
0
1
0
0
0
0
1
0
1
1
1
1
0
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
0
0
1
0
0
0
0
1
0
0
0
1
0
0
0
1
1
0
1
0
1
0
0
1
1
2
1
0
0
0
0
1
0
1
1
0
1
1
0
0
0
0
0
0
1
1
0
0
0
1
1
0
1
0
0
1
0
1
0
0
1
0
0
1
0
1
1
0
0
0
2
0
1
1
0
0
0
0
0
0
0
1
1
1
1
0
0
1
1
0
0
0
2
0
1
1
1
0
1
0
1
0
1
1
1
0
0
1
0
0
1
0
1
1
0
0
1
1
0
0
1
0
1
1
0
1
1
0
1
0
0
0
0
0
1
1
1
0
0
0
0
1
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
1
1
0
1
0
2
0
1
0
0
1
0
0
1
0
1
1
0
0
1
0
1
0
0


In [12]:
print(test_target)

30299    0
33736    1
46495    0
26529    0
27382    0
57499    0
39288    1
10627    1
46794    1
5224     2
7301     0
17986    0
46954    1
48766    0
14092    0
23288    0
53324    1
31747    0
52179    0
11049    0
9081     0
8056     0
14544    1
4621     0
56460    0
902      1
9316     1
45416    1
4592     1
35838    1
        ..
7772     0
52969    1
55365    1
11050    0
7191     1
32423    1
16857    0
23162    0
48961    1
31621    0
38933    1
16390    1
42637    0
14155    2
39826    1
44121    0
3310     1
32807    2
9301     0
42694    1
37789    1
44261    0
39674    1
1120     1
10395    1
28602    0
50350    0
5678     0
17680    0
37671    0
Name: status_group, Length: 5940, dtype: int64
